# Pegando dados da próxima corrida

In [4]:
import re
import os
import time
from datetime import datetime, timedelta


In [5]:
import cloudscraper
import pandas as pd
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
from selenium.webdriver.remote.webdriver import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from utils import london_time, establish_connection


# Configura as opções do Chrome para executar em modo headless
chrome_options = Options()
chrome_options.add_argument("--headless")


ModuleNotFoundError: No module named 'utils'

In [ ]:
# URLs
url_base = "https://www.oddschecker.com/greyhounds"
pattern = r"/greyhounds/[a-zA-Z-]+/\d{2}:\d{2}/winner"
top_2_finish = "top-2-finish"
top_3_finish = "top-3-finish"

# Query
query = "INSERT INTO odds (odd, nome_pista, quando, trap, nome_galgo, mercado) VALUES (%s, %s, %s, %s, %s, %s) ON CONFLICT (nome_pista, quando, trap, nome_galgo, mercado) DO UPDATE SET odd = excluded.odd"

In [ ]:

# Cria o scraper
scraper = cloudscraper.create_scraper()

# Busca as corridas da página
request = scraper.get(url_base)
request = request.text
# request


In [ ]:

soup = BeautifulSoup(request, "html.parser")
# print(soup.prettify())

In [ ]:

# Constrói um dataframe com as corridas do dia
def get_race_list(soup):
    # Verificar se a tag 'html' contém o atributo 'ng-app="ocAngularApp"'
    if soup.html.has_attr("ng-app") and soup.html["ng-app"] == "ocAngularApp":
        # print("O atributo ng-app='ocAngularApp' existe.")
        races = soup.find_all(
            "li", class_="group accordian-parent beta-body", attrs={"data-day": True}
        )

        for race in races:
            races = race.find_all("a")
            for link in races:
                link = link.get("href")
                if re.match(pattern, link):
                    link = "https://www.oddschecker.com" + link
                    data.append(link)

    else:
        # print("O atributo ng-app='ocAngularApp' não existe.\n")
        def find_race_meets_container(tag):
            return (
                tag.name == "div"
                and tag.has_attr("class")
                and "race-meets-container" in tag["class"]
            )

        uk_container = soup.find(find_race_meets_container)

        races = uk_container.find_all("div", class_="race-details")

        for race in races:
            races = race.find_all("a")
            for link in races:
                link = link.get("href")
                if re.match(pattern, link):
                    link = "https://www.oddschecker.com" + link
                    data.append(link)

    df = pd.DataFrame(data, columns=["link"])
    df["lugar"] = df["link"].apply(
        lambda link: re.search(r"/greyhounds/([^/]+)/\d{2}:\d{2}/", link).group(1)
    )
    df["quando"] = df["link"].apply(
        lambda link: re.search(r"/\d{2}:\d{2}/", link).group().strip("/")
    )
    df["quando"] = pd.to_datetime(df["quando"])
    df["mercado"] = df["link"].apply(
        lambda link: re.search(r"/winner", link).group().strip("/")
    )
    race_list = df.sort_values("quando")

    return race_list

# Busca o link da próxima corrida no horário londrino
def get_next_race(df):
    next_race = df[df["quando"] > london_time()].head(1)
    next_race = next_race.iloc[0]["link"]
    print("\nMonitorando a próxima corrida:\n", next_race, "\n")

    return next_race

# Busca as próximas corridas nos próximos x minutos
def get_upcoming_races(df, minutes):
    # Calcula o tempo limite (x minutos a partir da hora atual)
    time_limit = london_time() + timedelta(minutes=minutes)

    # Filtra as corridas cujo horário esteja dentro do intervalo
    upcoming_races = df[(df["quando"] > london_time()) & (df["quando"] <= time_limit)]
    upcoming_races_links = upcoming_races["link"].tolist()

    print("\n", upcoming_races_links, "\n")

    return upcoming_races_links

# Pega as odds da corrida
def get_data_races(race):
    # Extraindo dados da URL
    url_parts = race.split("/")

    onde = url_parts[4]
    quando = url_parts[5]
    mercado = url_parts[6]
    date = datetime.now().date()
    quando = f"{date} {quando}"

    try:
        request = scraper.get(race)
        request = request.text

        soup = BeautifulSoup(request, "html.parser")

        if soup.html.has_attr("ng-app") and soup.html["ng-app"] == "ocAngularApp":
            print("pega por lista <li>")
            # SITUAÇÃO 2
            driver = uc.Chrome()
            driver.get(race)

            # click on popup
            time.sleep(5)
            driver.find_element(
                By.XPATH, "/html/body/div[2]/div/div/div/button[1]"
            ).click()

            html_content = driver.page_source

            soup = BeautifulSoup(html_content, "html.parser")

            # # consulte o html
            # with open("race_list_list.html", "w") as file:
            #     file.write(soup.prettify())

            # Encontrar o tbody desejado pelo id
            tbody = soup.find("tbody", id="t1")

            data = []

            # Iterar sobre todos os elementos tr dentro do tbody
            for tr in tbody.find_all("tr", class_="diff-row evTabRow bc"):
                trap = tr.find("td", class_="trap-cell").text
                galgo = tr.find("td", class_="sel nm basket-active").text
                odds = tr.find("td", class_="bc")
                odd_frac = odds["data-o"]
                odd_dec = odds["data-odig"]
                data_fodds = odds["data-fodds"]

                # data.append([trap, galgo, odd_dec, onde, quando, mercado])
                data.append([trap, galgo, odd_dec, odd_frac, data_fodds, onde, quando, mercado])

                conn = establish_connection()
                cur = conn.cursor()
                query = "INSERT INTO odds (odd, nome_pista, quando, trap, nome_galgo, mercado) VALUES (%s, %s, %s, %s, %s, %s) ON CONFLICT (nome_pista, quando, trap, nome_galgo, mercado) DO UPDATE SET odd = excluded.odd"
                data_sql = (odd_dec, onde, quando, trap, galgo, mercado)
                cur.execute(query, data_sql)
                conn.commit()
                cur.close()
                conn.close()

            driver.quit()
        else:
            print("pega por div <div>")
            # SITUAÇÃO 1

            dog_list = soup.find_all("tr", class_="diff-row evTabRow bc")

            data = []

            for dog in dog_list:
                trap = dog.find("span", class_="trap").text
                galgo = dog.find("a", class_="popup selTxt").text
                odds = dog.find("td", class_="bc")
                odd_frac = odds["data-o"]
                odd_dec = odds["data-odig"]
                data_fodds = odds["data-fodds"]

                data.append(
                    [trap, galgo, odd_dec, odd_frac, data_fodds, onde, quando, mercado]
                )

                conn = establish_connection()
                cur = conn.cursor()
                query = "INSERT INTO odds (odd, nome_pista, quando, trap, nome_galgo, mercado) VALUES (%s, %s, %s, %s, %s, %s) ON CONFLICT (nome_pista, quando, trap, nome_galgo, mercado) DO UPDATE SET odd = excluded.odd"
                data_sql = (odd_dec, onde, quando, trap, galgo, mercado)
                cur.execute(query, data_sql)
                conn.commit()
                cur.close()
                conn.close()

            # print(data)
        try:
            df = pd.DataFrame(
                data,
                columns=[
                    "trap",
                    "galgo",
                    "odd_dec",
                    "odd_frac",
                    "data_fodds",
                    "onde",
                    "quando",
                    "mercado",
                ],
            )
            
            if not os.path.isfile('dados.csv'):
                df.to_csv('dados.csv', index=False)
            else:
                df_existing = pd.read_csv('dados.csv')
                df_updated = pd.concat([df_existing, df], ignore_index=True)
                df_updated.to_csv('dados.csv', index=False)
                
        except:
            print('Erro ao criar csv')

    except scraper.simpleException as e:
        print(e)

    print(df)
    return df



In [ ]:

race_list = get_race_list(soup)
print("\n Total de corridas hoje:", get_race_list.shape[0])


In [ ]:
while True:
    try:
        next_race = get_next_race(race_list)
        df_winner = get_data_races(next_race)

        top_2_finish_nxr = next_race.replace("winner", top_2_finish)
        top_3_finish_nxr = next_race.replace("winner", top_3_finish)

        top_2_finish_nxr = get_data_races(top_2_finish_nxr)
        top_3_finish_nxr = get_data_races(top_3_finish_nxr)

        time.sleep(30)

    except:
        print("Erro ao tentar capturar a próxima corrida")
        print("Provavelmente elas terminaram...")
        break